# Build a neural network with hyper-parameter fine tuning model

In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import keras_tuner as kt

In [2]:
# Load the MNIST dataset
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0  # Normalize pixel values to be between 0 and 1


In [3]:
# Define a hypermodel using Keras Tuner
class HyperModel(kt.HyperModel):
    def __init__(self, input_shape, num_classes):
        self.input_shape = input_shape
        self.num_classes = num_classes

    def build(self, hp):
        model = keras.Sequential()

        model.add(layers.Flatten(input_shape=self.input_shape))

        # Tune the number of units in the first dense layer
        hp_units = hp.Int('units', min_value=32, max_value=512, step=32)
        model.add(layers.Dense(units=hp_units, activation='relu'))
        
        # Tune the number of hidden layers
        hp_layers = hp.Int('num_layers', min_value=1, max_value=3)
        for _ in range(hp_layers):
            model.add(layers.Dense(units=hp_units, activation='relu'))

        model.add(layers.Dense(self.num_classes, activation='softmax'))

        # Tune the learning rate for the optimizer
        hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

        model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),
                      loss='sparse_categorical_crossentropy',
                      metrics=['accuracy'])

        return model

# Create an instance of the hypermodel
hypermodel = HyperModel(input_shape=(28, 28), num_classes=10)


In [4]:
# Initialize the tuner (e.g., RandomSearch or BayesianOptimization)
tuner = kt.Hyperband(hypermodel,
                     objective='val_accuracy',
                     max_epochs=10,
                     factor=3,
                     directory='my_dir',
                     project_name='mnist_tuning')


Reloading Tuner from my_dir\mnist_tuning\tuner0.json


In [5]:
# Search for the best hyperparameters
tuner.search(x_train, y_train, epochs=10, validation_data=(x_test, y_test))

# Get the best hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

# Build the model with the best hyperparameters
model = tuner.hypermodel.build(best_hps)

# Train the model with the best hyperparameters
model.fit(x_train, y_train, epochs=10, validation_data=(x_test, y_test))

Trial 10 Complete [00h 00m 33s]
val_accuracy: 0.9725000262260437

Best val_accuracy So Far: 0.9764000177383423
Total elapsed time: 00h 11m 35s

Search: Running Trial #11

Value             |Best Value So Far |Hyperparameter
160               |256               |units
2                 |2                 |num_layers
0.01              |0.001             |learning_rate
2                 |2                 |tuner/epochs
0                 |0                 |tuner/initial_epoch
2                 |2                 |tuner/bracket
0                 |0                 |tuner/round

Epoch 1/2
1875/1875 [==============================] - 11s 5ms/step - loss: 0.3233 - accuracy: 0.9126 - val_loss: 0.2509 - val_accuracy: 0.9403
Epoch 2/2
  20/1875 [..............................] - ETA: 10s - loss: 0.2074 - accuracy: 0.9578

KeyboardInterrupt: 